In [1]:
import pandas as pd
import numpy as np

In [2]:
# Import data
file_path = 'C:/Users/User/Documents/SKRIPSI/OPTIMASI PER PENYAKIT/Dataset_Skripsi_Processed.csv'  # Path absolut
df = pd.read_csv(file_path)

In [3]:
# Menampilkan data pertama untuk memastikan dataset terbaca dengan benar
pd.set_option('display.max_rows', None)
print(df)

    kode_penyakit kode_gejala  cf1  cf2  cf_median  cf_pakar
0             P01         G01  0.0  0.0        0.0       0.0
1             P01         G02  0.6  1.0        0.8       0.8
2             P01         G03  0.0  0.0        0.0       0.0
3             P01         G04  0.0  0.0        0.0       0.0
4             P01         G05  0.0  0.0        0.0       0.0
5             P01         G06  0.0  0.0        0.0       0.0
6             P01         G07  0.0  0.0        0.0       0.0
7             P01         G08  0.0  0.0        0.0       0.0
8             P01         G09  1.0  0.8        0.9       1.0
9             P01         G10  0.8  1.0        0.9       1.0
10            P01         G11  0.6  1.0        0.8       0.8
11            P01         G12  0.6  1.0        0.8       0.8
12            P01         G13  0.0  0.0        0.0       0.0
13            P01         G14  0.0  0.0        0.0       0.0
14            P01         G15  0.0  0.0        0.0       0.0
15            P01       

In [4]:
# Filter data untuk P01 saja
df_p01 = df[df['kode_penyakit'] == 'P01']  # Ambil data hanya untuk P01

In [5]:
# Menampilkan data yang diambil untuk P01
print("Data untuk P01:")
print(df_p01)

Data untuk P01:
   kode_penyakit kode_gejala  cf1  cf2  cf_median  cf_pakar
0            P01         G01  0.0  0.0        0.0       0.0
1            P01         G02  0.6  1.0        0.8       0.8
2            P01         G03  0.0  0.0        0.0       0.0
3            P01         G04  0.0  0.0        0.0       0.0
4            P01         G05  0.0  0.0        0.0       0.0
5            P01         G06  0.0  0.0        0.0       0.0
6            P01         G07  0.0  0.0        0.0       0.0
7            P01         G08  0.0  0.0        0.0       0.0
8            P01         G09  1.0  0.8        0.9       1.0
9            P01         G10  0.8  1.0        0.9       1.0
10           P01         G11  0.6  1.0        0.8       0.8
11           P01         G12  0.6  1.0        0.8       0.8
12           P01         G13  0.0  0.0        0.0       0.0
13           P01         G14  0.0  0.0        0.0       0.0
14           P01         G15  0.0  0.0        0.0       0.0
15           P01        

In [6]:
# Set seed for reproducibility
np.random.seed(42)

## Inisialisasi Parameter

In [7]:
# Parameter PSO tetap
w_max = 0.9  
w_min = 0.2
c1 = 1.0
c2 = 1.5
n_iterations = 10  # Jumlah iterasi

## Fungsi Fitness

In [8]:
# Fungsi fitness untuk mengukur sejauh mana posisi partikel mendekati cf_pakar
def fitness(position, cf_pakar):
    return np.mean((cf_pakar - position) ** 2, axis=1) 

## Implementasi

In [9]:
# Fungsi untuk menjalankan PSO dengan parameter bobot inersia yang berbeda untuk setiap penyakit
def run_pso(cf_pakar, n_particles):
    fitness_best_results = []
    
    for trial in range(1, 11):  # 10 uji coba
        # Inisialisasi kecepatan dan posisi partikel secara acak
        position = np.random.uniform(0.0, 1.0, (n_particles, len(cf_pakar)))  # Posisi partikel
        velocity = np.random.uniform(-0.1, 0.1, (n_particles, len(cf_pakar)))  # Kecepatan partikel

        # Inisialisasi pbest dan gbest
        pbest = position.copy()  # Set pbest awal sama dengan posisi partikel
        pbest_score = np.array([np.mean(np.abs(cf_pakar - p)) for p in position])  # Nilai fitness awal untuk pbest
        gbest = pbest[np.argmin(pbest_score)]  # gbest adalah posisi dengan nilai fitness terbaik
        gbest_score = np.min(pbest_score)  # Nilai fitness terbaik untuk gbest

        # Iterasi PSO
        for iteration in range(n_iterations):
            w = w_max - (w_max - w_min) * (iteration / n_iterations)  # Interpolasi w antara w_max dan w_min

            # Update posisi dan kecepatan setiap partikel
            for i in range(n_particles):
                r1 = np.random.rand(len(cf_pakar))  # Random number untuk C1
                r2 = np.random.rand(len(cf_pakar))  # Random number untuk C2

                # Update kecepatan
                velocity[i] = w * velocity[i] + c1 * r1 * (pbest[i] - position[i]) + c2 * r2 * (gbest - position[i])

                # Update posisi
                position[i] += velocity[i]

                # Pastikan posisi berada dalam rentang 0 dan 1
                position[i] = np.clip(position[i], 0.0, 1.0)

            # Evaluasi fitness setiap partikel
            score = fitness(position, cf_pakar)

            # Update pbest dan gbest
            for i in range(n_particles):
                if score[i] < pbest_score[i]:
                    pbest[i] = position[i]  # Update pbest dengan posisi partikel yang lebih baik
                    pbest_score[i] = score[i]

            # Update gbest jika ditemukan pbest yang lebih baik
            gbest_score_new = np.min(pbest_score)
            if gbest_score_new < gbest_score:
                gbest = pbest[np.argmin(pbest_score)]  # Update gbest dengan posisi pbest terbaik
                gbest_score = gbest_score_new

        # Simpan hasil fitness terbaik untuk setiap uji coba
        fitness_best_results.append(gbest_score)

    # Mengembalikan hasil fitness terbaik untuk 10 uji coba
    return fitness_best_results

In [10]:
# Daftar penyakit yang ingin diuji (hanya P01 untuk sekarang)
penyakit_codes = ['P01']  # Fokus pada P01 saja

In [11]:
# Menyimpan hasil uji coba per penyakit
results = []

In [12]:
# Kombinasi ukuran swarm yang diuji
n_particles_values = [5, 10, 15, 20, 25, 30, 35, 40, 45, 50, 55, 60, 65, 70, 75, 80, 85, 90, 95, 100, 105]

In [13]:
# Fokus pada P01
for penyakit in penyakit_codes:
    cf_pakar = df[df['kode_penyakit'] == penyakit]['cf_pakar'].values
    for n_particles in n_particles_values:
        # Jalankan PSO dan simpan hasil per percobaan
            fitness_results = run_pso(cf_pakar, n_particles)
            
            # Simpan hasil per percobaan (trial) untuk n_particles yang diuji
            for trial_num, fitness_score in enumerate(fitness_results, 1):
                results.append({
                    'Penyakit': penyakit,
                    'n_particles': n_particles,
                    'Percobaan': trial_num,
                    'Fitness Terbaik': fitness_score
                })
            

In [14]:
# Membuat DataFrame untuk menampilkan hasil
df_results = pd.DataFrame(results)

pd.set_option('display.max_rows', 1000)

In [15]:
# Menampilkan hasil tabel per percobaan
print(df_results)

    Penyakit  n_particles  Percobaan  Fitness Terbaik
0        P01            5          1         0.149005
1        P01            5          2         0.200943
2        P01            5          3         0.169793
3        P01            5          4         0.201511
4        P01            5          5         0.199164
5        P01            5          6         0.198768
6        P01            5          7         0.234225
7        P01            5          8         0.129635
8        P01            5          9         0.153158
9        P01            5         10         0.186184
10       P01           10          1         0.127700
11       P01           10          2         0.118188
12       P01           10          3         0.092329
13       P01           10          4         0.054909
14       P01           10          5         0.139765
15       P01           10          6         0.157194
16       P01           10          7         0.114674
17       P01           10   

In [16]:
# Menghitung rata-rata fitness terbaik untuk setiap kombinasi ωmax dan ωmin
avg_fitness_results = df_results.groupby(['Penyakit', 'n_particles'])['Fitness Terbaik'].mean().reset_index()

In [17]:
# Menampilkan kombinasi terbaik per penyakit berdasarkan rata-rata fitness
print("\nKombinasi n_particles terbaik per penyakit:")
print(avg_fitness_results)


Kombinasi n_particles terbaik per penyakit:
   Penyakit  n_particles  Fitness Terbaik
0       P01            5         0.182239
1       P01           10         0.106832
2       P01           15         0.087007
3       P01           20         0.067398
4       P01           25         0.072795
5       P01           30         0.059794
6       P01           35         0.054596
7       P01           40         0.043144
8       P01           45         0.045725
9       P01           50         0.030194
10      P01           55         0.026087
11      P01           60         0.024911
12      P01           65         0.031548
13      P01           70         0.034342
14      P01           75         0.025883
15      P01           80         0.028554
16      P01           85         0.020171
17      P01           90         0.029477
18      P01           95         0.027927
19      P01          100         0.016438
20      P01          105         0.010947


In [18]:
# Jika ingin menambahkan header dan data ke file yang sudah ada (append):
df_results.to_csv('pengujian_ukuran_swarm_P01.csv', mode='a', header=False, index=False)
